In [11]:
from ultralytics import YOLO
from PIL import Image
import numpy as np
import os


model = YOLO("yolov8x.pt")

In [145]:
# calculate the IoU
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou


# used to read all the txt file in a folder
def read_txt_files(folder_path):
    # Get a list of all .txt file paths in the folder
    txt_file_paths = [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith('.txt')
    ]
    
    # Sort the list of file paths alphabetically by filename
    txt_file_paths.sort(key=lambda x: os.path.basename(x))
    
    content_list = []  # Initialize an empty list to store file contents
    
    # Read the contents of each file in sorted order
    for file_path in txt_file_paths:
        with open(file_path, 'r') as file:
            file_content = file.read()  # Read the entire content of the file
            if file_content == '':
                content_list.append(file_content)  # Append the content to the list
            else :
                string = file_content[3:-2]
                string_representation = string.replace(' ', ', ')
                content_list.append(string_representation)
    return content_list

def read_img_files_size(folder_path):
    # Get a list of all .PNG file paths in the folder
    img_file_paths = [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith('.png')
    ]
    
    # Sort the list of file paths alphabetically by filename
    img_file_paths.sort(key=lambda x: os.path.basename(x))
    
    # List to store tuples of (file_path, image_size)
    image_data_list = []
    
    for file_path in img_file_paths:
        try:
            # Open the image file using PIL
            with Image.open(file_path) as img:
                # Get the dimensions (width, height) of the image
                image_size = img.size  # This will return a tuple (width, height)
                # Append (image_size) tuple to the list
                image_data_list.append(image_size)
        except Exception as e:
            print(f"Error processing image {file_path}: {e}")
    
    return image_data_list

# change yolo (x,y,w,h) bbox to bbox (x1, y1, x2, y2)
def yolobbox2bbox(x,y,w,h,img_w,img_h):
    w = w * img_w
    h = h * img_h
    x = x * img_w
    y = y * img_h
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    x1 = round(x1, 5)
    y1 = round(y1, 5)
    x2 = round(x2, 5)
    y2 = round(y2, 5)
    return [x1, y1, x2, y2]

# the list from yolo predict and ground truth are a string list
# change it to float
def string_to_float_list(list_string):
    # Step 1: Split the string by commas to separate values
    values_str = list_string.split(',')
    # Step 2: Convert each split string to a float
    values_float = [float(val.strip()) for val in values_str]
    # Step 3: Return the list of float values
    return values_float


# results can be obtain from yolo predict
def list_of_predicted(results):
    labels = []
    for result in results: 
        boxes = result.boxes  # Boxes object for bounding box outputs 

        # Convert each element in the array to a formatted string
        formatted_strings = [f"{value:.4f}" for value in boxes.xyxy.flatten()]

        # Join the formatted strings into a single string separated by commas
        string_representation = ", ".join(formatted_strings)
        labels.append(string_representation)
        
    return labels

In [146]:
# Usage
ground_truth = read_txt_files('dataset/labels/test')
imgsize = read_img_files_size('dataset/images/test')

labels = list_of_predicted(results)

In [158]:
frame = 467

gt = string_to_float_list(labels[frame])
temp = string_to_float_list(ground_truth[frame])
res = yolobbox2bbox(temp[0], temp[1], temp[2], temp[3], imgsize[frame][0], imgsize[frame][1])
print(res)
print(gt)

[1150.40064, 410.64192, 1184.66112, 443.39832]
[1152.3997, 411.989, 1183.5419, 441.1953]


In [156]:
bb_intersection_over_union(res, gt)

0.8156968514534586

In [7]:
results = model.predict(source="dataset/images/test", save=True, classes=32) #classes=32



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000000.PNG: 480x640 (no detections), 3691.4ms
image 2/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000001.PNG: 480x640 (no detections), 2895.8ms
image 3/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000002.PNG: 480x640 (no detections), 3898.2ms
image 4/1175 /Users/arvinn/Documents

image 56/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000055.PNG: 480x640 (no detections), 6107.9ms
image 57/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000056.PNG: 480x640 (no detections), 7942.2ms
image 58/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000057.PNG: 480x640 (no detections), 6671.2ms
image 59/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000058.PNG: 480x640 (no detections), 4252.4ms
image 60/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000059.PNG: 480x640 (no detections), 3927.9ms
image 61/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000060.PNG: 480x640 (no detections), 3518.6ms
image 62/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000061.PNG: 480x640 (no detections), 3510.9ms
image 63/1175

image 115/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000114.PNG: 480x640 (no detections), 3115.5ms
image 116/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000115.PNG: 480x640 (no detections), 3105.1ms
image 117/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000116.PNG: 480x640 (no detections), 2879.6ms
image 118/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000117.PNG: 480x640 (no detections), 3038.0ms
image 119/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000118.PNG: 480x640 (no detections), 2970.5ms
image 120/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000119.PNG: 480x640 (no detections), 3022.0ms
image 121/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000120.PNG: 480x640 (no detections), 3256.3ms
image 

image 173/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000172.PNG: 480x640 (no detections), 3344.7ms
image 174/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000173.PNG: 480x640 (no detections), 3900.5ms
image 175/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000174.PNG: 480x640 (no detections), 3583.7ms
image 176/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000175.PNG: 480x640 (no detections), 3170.9ms
image 177/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000176.PNG: 480x640 (no detections), 3391.2ms
image 178/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000177.PNG: 480x640 (no detections), 3075.6ms
image 179/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000178.PNG: 480x640 (no detections), 3408.6ms
image 

image 231/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000230.PNG: 480x640 (no detections), 2862.3ms
image 232/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000231.PNG: 480x640 (no detections), 2860.5ms
image 233/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000232.PNG: 480x640 (no detections), 2854.5ms
image 234/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000233.PNG: 480x640 (no detections), 2865.3ms
image 235/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000234.PNG: 480x640 (no detections), 2886.7ms
image 236/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000235.PNG: 480x640 (no detections), 2858.4ms
image 237/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000236.PNG: 480x640 (no detections), 2838.1ms
image 

...

image 931/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000930.PNG: 384x640 (no detections), 2948.2ms
image 932/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000931.PNG: 384x640 (no detections), 2651.8ms
image 933/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000932.PNG: 384x640 (no detections), 2397.3ms
image 934/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000933.PNG: 384x640 (no detections), 2552.2ms
image 935/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000934.PNG: 384x640 (no detections), 2310.7ms
image 936/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000935.PNG: 384x640 (no detections), 2342.9ms
image 937/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000936.PNG: 384x640 (no detections), 2351.0ms
image 

image 990/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000989.PNG: 384x640 (no detections), 2300.3ms
image 991/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000990.PNG: 384x640 (no detections), 2301.1ms
image 992/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000991.PNG: 384x640 (no detections), 2383.5ms
image 993/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000992.PNG: 384x640 (no detections), 2436.3ms
image 994/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000993.PNG: 384x640 (no detections), 2634.3ms
image 995/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000994.PNG: 384x640 (no detections), 2366.8ms
image 996/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000995.PNG: 384x640 (no detections), 2309.1ms
image 

image 1048/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001047.PNG: 384x640 (no detections), 2300.1ms
image 1049/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001048.PNG: 384x640 (no detections), 2318.7ms
image 1050/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001049.PNG: 384x640 (no detections), 2314.5ms
image 1051/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001050.PNG: 384x640 (no detections), 2301.1ms
image 1052/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001051.PNG: 384x640 (no detections), 2317.0ms
image 1053/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001052.PNG: 384x640 (no detections), 2294.2ms
image 1054/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001053.PNG: 384x640 (no detections), 2355.0ms

image 1106/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001105.PNG: 384x640 (no detections), 3002.6ms
image 1107/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001106.PNG: 384x640 (no detections), 3079.3ms
image 1108/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001107.PNG: 384x640 (no detections), 3235.5ms
image 1109/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001108.PNG: 384x640 (no detections), 3348.1ms
image 1110/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001109.PNG: 384x640 (no detections), 3155.2ms
image 1111/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001110.PNG: 384x640 (no detections), 4226.2ms
image 1112/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001111.PNG: 384x640 (no detections), 3209.5ms

image 1164/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001163.PNG: 384x640 (no detections), 2830.4ms
image 1165/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001164.PNG: 384x640 (no detections), 2849.5ms
image 1166/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001165.PNG: 384x640 (no detections), 2904.6ms
image 1167/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001166.PNG: 384x640 (no detections), 2897.0ms
image 1168/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001167.PNG: 384x640 (no detections), 3013.7ms
image 1169/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001168.PNG: 384x640 (no detections), 3622.1ms
image 1170/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001169.PNG: 384x640 (no detections), 2971.6ms